# Laboratorio 04 | The Games of Nodes Part II

Agregaciones en Neo4j (Cypher) usando Count, Collect y Size

## Configuración del entorno

In [12]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()

uri =  os.getenv("NEO4J_URI")         
username = os.getenv("NEO4J_USERNAME")                      
password = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(username, password))

def close_connection():
    driver.close()

def verify_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1 AS value")
            value = result.single()["value"]
            print("Connection successful. Result:", value)
    except Exception as e:
        print("Connection failed:", e)

### Verificar la conexión

In [10]:
verify_connection()
close_connection()


Connection successful. Result: 1


C:\Users\diego\AppData\Local\Temp\ipykernel_10872\2394711248.py:14: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


# 🔎 Acto 01: El Descenso de los Cineastas
Somos detectives de datos en NeoCity, donde cada persona, lugar y evento está conectado en un vasto grafo. Últimamente, la comunidad de cineastas ha visto una caída en el número de directores activos, y nos han contratado para averiguar por qué.

### **Investigación 1:** Determinar la cantidad de cineastas antes y después de 2010
- **Objetivo:** Determinar cuántos cineastas han ingresado a la industria antes y después del 2010.

In [22]:
def get_arround_2010():
  query1 = """
    MATCH (p:Person)
    WHERE p.yearJoined < 2010
    RETURN COUNT(p) AS CineastasAntes
  """
  query2 = """
    MATCH (p:Person)
    WHERE p.yearJoined >= 2010
    RETURN COUNT(p) AS CineastasDespues
  """
  with driver.session() as session:
    result1 = session.run(query1)
    cineastas_antes = result1.single()["CineastasAntes"]
    
    result2 = session.run(query2)
    cineastas_despues = result2.single()["CineastasDespues"]
    
    return cineastas_antes, cineastas_despues

cineastas_antes_2010, cineastas_despues_2010 = get_arround_2010()
print("Cineastas antes de 2010:", cineastas_antes_2010)
print("Cineastas después de 2010:", cineastas_despues_2010)

Cineastas antes de 2010: 6
Cineastas después de 2010: 4


>Pista revelada: Se ha determinado una notable disminución en la contratación de nuevos cineastas después de 2010.

### **Investigación 2:** Analizar los cambios en la industria con base en los géneros
- **Objetivo:** Investigar los cambios en la industria que podrían estar relacionados con los géneros de las películas. Uso de COUNT().

In [28]:
def get_movie_genres_count():
  query = """
  MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
  RETURN g.name AS Genero, COUNT(m) AS NumPeliculas
  ORDER BY NumPeliculas DESC
  """
  with driver.session() as session:
    result = session.run(query)
    genres_count = [{"Genero": record["Genero"], "NumPeliculas": record["NumPeliculas"]} for record in result]
    return genres_count

movie_genres_count = get_movie_genres_count()

for genre in movie_genres_count:
    print(f"Genero: {genre['Genero']}, NumPeliculas: {genre['NumPeliculas']}")

Genero: Drama, NumPeliculas: 6
Genero: Acción, NumPeliculas: 6
Genero: Suspenso, NumPeliculas: 5
Genero: Ciencia Ficción, NumPeliculas: 4
Genero: Terror, NumPeliculas: 3
Genero: Comedia, NumPeliculas: 1


>Pista revelada: Se observa que algunos géneros, como el suspenso y la acción, han visto un aumento en producción, mientras que otros han decaído.

###  **Investigación 3:** ¿Cómo ha afectado la disminución de cineastas a las colaboraciones?
- Objetivo: Investigar si la disminución de cineastas ha afectado la colaboración dentro de la comunidad. Uso de COUNT().

In [29]:
def get_collaborations_count():
  query = """
  MATCH (p:Person)-[:COLLABORATED_WITH]->(colleague)
  RETURN COUNT(DISTINCT p) AS CineastasColaborando
  """
  with driver.session() as session:
    result = session.run(query)
    collaborations_count = result.single()["CineastasColaborando"]
    return collaborations_count

collaborations_count = get_collaborations_count()
print("Colaboraciones entre cineastas:", collaborations_count)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: COLLABORATED_WITH)} {position: line: 2, column: 22, offset: 22} for query: '\n  MATCH (p:Person)-[:COLLABORATED_WITH]->(colleague)\n  RETURN COUNT(DISTINCT p) AS CineastasColaborando\n  '


Colaboraciones entre cineastas: 0


>Pista revelada: Las colaboraciones han aumentado, lo que sugiere que la comunidad cinematográfica es más unida y competitiva que antes.